# Constraint-based sampling for metabolic flux analysis (MFA)

Here we jump into the (for now) final step in our metabolic flux analysis (MFA) pipeline. After constructing an INCA script, running it in MATLAB and reimporting the data we're now here. This example notebook will guide you through different ways to integrate your MFA results into COBRA models and how to make them more reliable.

In [1]:
import pandas as pd
import numpy as np
import time
import ast
import sys
import escher
import cobra
from gurobipy import Model as GRBModel
import re
from BFAIR.INCA import INCA_reimport
from BFAIR.INCA.sampling import *

Academic license - for non-commercial use only - expires 2021-05-28
Using license file /Users/matmat/gurobi.lic


#### INCA re-import

First, let's reimport the data using our `BFAIR INCA_reimport` tools

The data is taken from a different folder

In [2]:
%cd /Users/matmat/Documents/Random data/Stefano
filename = 'data/MFA_modelInputsData/TestFile.mat'

/Users/matmat/Documents/Random data/Stefano


Then we move back to our example folder to get the remaining input

In [3]:
%cd /Users/matmat/Documents/GitHub/AutoFlow-OmicsDataHandling/examples
#filename = 'data/MFA_modelInputsData/TestFile.mat'
simulation_info = pd.read_csv('data/MFA_modelInputsData/Re-import/experimentalMS_data_I.csv')
simulation_id = 'WTEColi_113C80_U13C20_01'

/Users/matmat/Documents/GitHub/AutoFlow-OmicsDataHandling/examples


Here we re-import the INCA output

In [4]:
reimport_data = INCA_reimport()
(fittedData,
 fittedFluxes,
 fittedFragments,
 fittedMeasuredFluxes,
 fittedMeasuredFragments,
 fittedMeasuredFluxResiduals,
 fittedMeasuredFragmentResiduals,
 simulationParameters) = reimport_data.reimport(
    filename,
    simulation_info,
    simulation_id
)

Here we import the model

In [5]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')

In [6]:
original_solution = model.optimize()
original_solution

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000e+00
EX_cmp_e,0.000000,-2.965572e-01
EX_co2_e,19.675223,0.000000e+00
EX_cobalt2_e,-0.000025,-0.000000e+00
DM_4crsol_c,0.000219,0.000000e+00
...,...,...
RNDR4,0.000000,-2.073827e-03
RNDR4b,0.000000,-2.073827e-03
RNTR1c2,0.025705,-8.673617e-18
RNTR2c2,0.026541,-8.673617e-18


Let's see what happens now when we add our new bound constraints

In [7]:
model = add_constraints(model, fittedFluxes)
model.optimize()

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpsl65z84j.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpgewjdjph.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
add_constraints takes 0h: 0min: 11sec to run
--- end ---


/Users/matmat/opt/anaconda3/envs/bfair/lib/python3.8/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


<Solution infeasible at 0x7fb494cac550>

Oh... That's not good... Well that sucks, seems like we have to deal with an infeasible solution. There are two straight forward ways: exclusion and relaxation

#### Dealing with infeasible solutions - exclusion

The easier way to deal with this issue is to simply exclude the constraints that render a model infeasible. We can do that by adding the calculated bounds one by one. If we come across a reaction whose bounds cause trouble, we restart the process and skip this one. This might have to be done a few times to exclude all troublemakers. the `add_feasible_constraints()` functions takes care of that for us. Let's reset the model first.

In [8]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')

In [9]:
find_biomass_reaction(model)

['BIOMASS_Ec_iJO1366_WT_53p95M', 'BIOMASS_Ec_iJO1366_core_53p95M']

In [13]:
min_val = get_min_solution_val(fittedFluxes, biomass_string='Biomass')

In [11]:
replace_biomass_rxn_name(fittedFluxes, biomass_string='Biomass', biomass_rxn_name='BIOMASS_Ec_iJO1366_core_53p95M')

In [14]:
model, problems = add_feasible_constraints(model, fittedFluxes, min_val=min_val)

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp0dg50cyi.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpx6hi8h7k.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


/Users/matmat/opt/anaconda3/envs/bfair/lib/python3.8/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


Solution infeasible if adding ASPTA
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpyin8zcjq.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp0ax_zrjv.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding BIOMASS_Ec_iJO1366_core_53p95M
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmppqzd0dn0.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp7ngwle4n.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
---------------------------
Total number of restarts: 2
add_feasible_constraints takes 0h: 0min: 35sec to run
--- end ---


The `model` is our newly constrained model and the problematic reactions can be listed in `problems`.

In [15]:
model

Name,iJO1366
Memory address,0x07fb49e444700
Number of metabolites,1805
Number of reactions,2583
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


In [16]:
problems

['ASPTA', 'BIOMASS_Ec_iJO1366_core_53p95M']

Now let's see what an effect these new bounds had on the predicted growth rate (the objective value) of our model

In [17]:
new_bounds_solution = model.optimize()
new_bounds_solution

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000
EX_cmp_e,0.000000,-1.914094
EX_co2_e,14.057895,0.000000
EX_cobalt2_e,0.000000,-0.000000
DM_4crsol_c,0.000000,0.000000
...,...,...
RNDR4,0.000000,0.000000
RNDR4b,0.000000,0.000000
RNTR1c2,0.000000,-1.914094
RNTR2c2,0.000000,-1.914094


And here's the star of the show, our sampling method. We trust our models because... we have to! And because smart people that knew what they were doing set them up. So in order to gain more confidence in our MFA data, we sample the model after adding the calculated bound for some of the reactions and re-calculate the fluxes a number of time. Then, we take the mean and take that as the most trustworthy calculated flux. These fluxes can be visualized, for example in tools like `Escher`

In [18]:
sampled_fluxes = cobra.sampling.sample(model, n=100, processes=2)

Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpurhq2m_u.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpl0_7thtn.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


In [19]:
sampled_fluxes

,EX_cm_e,EX_cmp_e,EX_co2_e,EX_cobalt2_e,DM_4crsol_c,DM_5drib_c,DM_aacald_c,DM_amob_c,DM_mththf_c,EX_colipa_e,...,UREAtex,RNDR2b,UREAtpp,RNDR3,RNDR3b,RNDR4,RNDR4b,RNTR1c2,RNTR2c2,RNTR3c2
0,0.0,0.0,27.751707,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.001413,3.555913e-21,-0.001413,0.0,0.0,3.787820e-21,0.0,-6.598197e-21,3.787820e-21,6.598197e-21
1,0.0,0.0,27.748104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.000176,5.651052e-21,-0.000176,0.0,0.0,6.019599e-21,0.0,-1.048585e-20,6.019599e-21,1.048585e-20
2,0.0,0.0,27.546434,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.000486,1.013825e-17,-0.000486,0.0,0.0,1.079944e-17,0.0,-1.881209e-17,1.079944e-17,1.881209e-17
3,0.0,0.0,27.549097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.000557,1.013802e-17,-0.000557,0.0,0.0,1.079920e-17,0.0,-1.881167e-17,1.079920e-17,1.881167e-17
4,0.0,0.0,27.542983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.000557,1.013780e-17,-0.000557,0.0,0.0,1.079896e-17,0.0,-1.881125e-17,1.079896e-17,1.881125e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,23.970243,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.033269,-1.323152e-15,-0.033269,0.0,0.0,2.468379e-15,0.0,-3.372211e-15,1.314471e-15,3.372211e-15
96,0.0,0.0,24.074182,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.033287,-1.323791e-15,-0.033287,0.0,0.0,2.469904e-15,0.0,-3.375074e-15,1.315228e-15,3.375074e-15
97,0.0,0.0,24.010473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.039541,-1.322980e-15,-0.039541,0.0,0.0,2.468044e-15,0.0,-3.371723e-15,1.314295e-15,3.371723e-15
98,0.0,0.0,23.951884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.039538,-1.322895e-15,-0.039538,0.0,0.0,2.467859e-15,0.0,-3.371406e-15,1.314201e-15,3.371406e-15


In [20]:
fluxes_sampling = reshape_fluxes_escher(sampled_fluxes)

In [21]:
sampled_flux_map = escher.Builder('e_coli_core.Core metabolism',
                                  reaction_data = fluxes_sampling).display_in_notebook()
sampled_flux_map

There are also other ways to cosolidate the MFA calculations and the constraint based flux predictions. One of these is lMOMA (linear Minimization Of Metabolic Adjustment). MOMA assumes that the fluxes before and after adding the new constraints should be similar, so it aims to predicting an optimum for the newly constrined model while keeping the differences to the original model small. We suggest using pFBA (parsimonious Flux Balance Analysis) instead of regular FBA for this step as pFBA aims to keep the overal fluxes low.

In [22]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')
model, problems = add_feasible_constraints(model, fittedFluxes, min_val=min_val)

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpriraj84k.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpf4oyg30j.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding ASPTA
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpz0wb74z4.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


/Users/matmat/opt/anaconda3/envs/bfair/lib/python3.8/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpmx9igl1l.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding BIOMASS_Ec_iJO1366_core_53p95M
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpib7hk4nm.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpdu8zll87.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
---------------------------
Total number of restarts: 2
add_feasible_constraints takes 0h: 0min: 36sec to run
--- end ---


In [23]:
pfba_solution = cobra.flux_analysis.pfba(model)

In [24]:
moma_after_MFA = cobra.flux_analysis.moma(model=model, solution=pfba_solution, linear=True)

For some reason I get weird values when I just look at the objective value here, both for pFBA and for lMOMA, so I just look at the biomass function instead. This should produce a similar result as the regular FBA prediction after adding the new constraints

In [25]:
moma_after_MFA

,fluxes,reduced_costs
EX_cm_e,0.000000,0.0
EX_cmp_e,0.000000,0.0
EX_co2_e,4.486364,0.0
EX_cobalt2_e,0.000000,0.0
DM_4crsol_c,0.000000,0.0
...,...,...
RNDR4,0.000000,0.0
RNDR4b,0.000000,0.0
RNTR1c2,0.000000,0.0
RNTR2c2,0.000000,0.0


In [26]:
moma_after_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]

0.0

In [27]:
print(f'Before adding new constraints:             {original_solution.objective_value:.2f}')
print(f'After adding new constraints:              {new_bounds_solution.objective_value:.2f}')
print(f'After adding new constraints + pFBA, objV: {pfba_solution.objective_value:.2f}')
print(f'After adding new constraints + pFBA, BM:   {pfba_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]:.2f}')
print(f'After adding new constraints + MOMA, objV: {moma_after_MFA.objective_value:.2f}')
print(f'After adding new constraints + MOMA, BM:   {moma_after_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]:.2f}')

Before adding new constraints:             0.98
After adding new constraints:              0.00
After adding new constraints + pFBA, objV: 193.26
After adding new constraints + pFBA, BM:   0.00
After adding new constraints + MOMA, objV: 0.00
After adding new constraints + MOMA, BM:   0.00


Of course also the MOMA results can be visualized with `Escher`. The `reshape_fluxes_escher()` can take both pandas DataFrames or cobra solutions as an input.

In [28]:
fluxes_moma = reshape_fluxes_escher(moma_after_MFA)

In [29]:
moma_flux_map = escher.Builder('e_coli_core.Core metabolism',
                               reaction_data = fluxes_moma).display_in_notebook()
moma_flux_map

#### Dealing with infeasible solutions - relaxation

Another way of dealing with infeasible models is to relax the added constraints to the point that it works again. You will need to have the Gurobi solver installed for this. The same principle is used in the `BFAIR thermo` tools. For that we add our constraints to a model that will now be infeasible. Have I meantioned that this is much more elegant, better and that you should do that? It is. The other method is *fine* but you exclude reactions and, in general, it is always better to use as much as possible of the information that is available to you.

In [30]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')

In [31]:
model = add_constraints(model, fittedFluxes)

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpec0nvo57.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpgsefgext.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
add_constraints takes 0h: 0min: 10sec to run
--- end ---


In [32]:
model.optimize()

<Solution infeasible at 0x7fb433994760>

Then we make use of the handy `bound_relaxation()` function that will test our model to figure out which of these added bounds need to be adjusted and return a DataFrame that describes the affected functions and the gravity of the suggested changes. If we allow this function to be `desctructive` it will adjust the input model right away.

In [34]:
bound_relaxation(model, fittedFluxes, destructive=True, fluxes_to_ignore=['BIOMASS_Ec_iJO1366_core_53p95M'])

--- start ---
relax_obj: 6.766453568750002
bound_relaxation takes 0h: 0min: 0sec to run
--- end ---


,lb_change,ub_change,subsystem
reaction,,,
ASPTA,-3.349375,0.000000,Alanine and Aspartate Metabolism
DAPDC_reverse_d3ab8,-0.040213,0.000000,Threonine and Lysine Metabolism
GLNS_reverse_59581,-0.767431,0.000000,Glutamate Metabolism
MTHFC_reverse_f6fcc,-0.515687,0.000000,Folate Metabolism
MTHFD_reverse_c10fd,-0.515687,0.000000,Folate Metabolism
RPI_reverse_853a1,0.000000,1.578062,Pentose Phosphate Pathway


Let's see if our model is feasible now.

In [35]:
relaxed_solution = model.optimize()
relaxed_solution

,fluxes,reduced_costs
EX_cm_e,0.000000,0.0
EX_cmp_e,0.000000,-0.0
EX_co2_e,20.755531,0.0
EX_cobalt2_e,-0.000017,-0.0
DM_4crsol_c,0.000156,0.0
...,...,...
RNDR4,0.000000,0.0
RNDR4b,0.000000,0.0
RNTR1c2,0.018316,0.0
RNTR2c2,0.018912,0.0


In [36]:
relaxed_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]

0.7

And now we're also at the point where we can let the power of constraint based models work for us and make use of the tools mentioned above in order to adjust the fluxes calculated using MFA so that they nicely fit into our model.

In [37]:
relaxed_sampled_fluxes = cobra.sampling.sample(model, n=100, processes=2)

Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpwfflaf63.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpfw5eeg1j.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


In [38]:
relaxed_fluxes_sampling = reshape_fluxes_escher(relaxed_sampled_fluxes)

In [39]:
relaxed_flux_map = escher.Builder('e_coli_core.Core metabolism',
                                  reaction_data = relaxed_fluxes_sampling).display_in_notebook()
relaxed_flux_map